## Graph dependency


In [4]:
# import essentials libraries
import nltk
from nltk.corpus import stopwords
import numpy as np 
import os 
import pandas as pd 
import re
import spacy
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd 
from nltk.corpus import stopwords 
from wordcloud import WordCloud, STOPWORDS
import math 
import statistics 
from sklearn.feature_extraction.text import CountVectorizer 
from textblob import TextBlob
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
import os 
import pandas as pd
from transformers import pipeline
import torch
import time
from datetime import timedelta
import psutil

import re

[nltk_data] Downloading package stopwords to C:\Users\Pin
[nltk_data]     Nean\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Pin
[nltk_data]     Nean\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1. concatenate all reviews from all sources (youtube, ebay, productreviews)

In [5]:
data_folder = "data" 
data_filename = "product_review_heater4.csv"
data_filenames = ["product_review_heater1.csv","product_review_heater2.csv","product_review_heater3.csv","product_review_heater4.csv"]
dfs = []

for data_filename in data_filenames: 
    data_filepath = os.path.join(data_folder, data_filename)
    df = pd.read_csv(data_filepath)
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df['Reviews'] = df['Reviews'].dropna().apply(lambda s: eval(s))
    df = df.dropna()
    dfs.append(df)
    
df_pr = pd.concat(dfs)


In [6]:
data_folder = "support\heater_appliance"
data_filename = "combined_data.pkl"
data_filepath = os.path.join(data_folder, data_filename)

df_yt = pd.DataFrame(pd.read_pickle(data_filepath))
df_yt.columns = ["Title", "Page", "Reviews"]


In [7]:
data_folder = "support\heater_product"
data_filename = "combined_data.pkl"
data_filepath = os.path.join(data_folder, data_filename)

df_yt2 = pd.DataFrame(pd.read_pickle(data_filepath))
df_yt2.columns = ["Title", "Page", "Reviews"]

In [8]:
data_folder = "heater_data"
# data_filename = "product_review_heater4.csv"
data_filenames = os.listdir(data_folder)
dfs = []

for data_filename in data_filenames: 
    data_filepath = os.path.join(data_folder, data_filename)
    df = pd.read_csv(data_filepath)
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df['Reviews'] = df['Reviews'].dropna().apply(lambda s: eval(s))
    df = df.dropna(subset=['Reviews'])
    dfs.append(df)
    


    
df_uncleaned = pd.concat(dfs)
df_uncleaned.columns = map(str.strip,df_uncleaned.columns)
df_uncleaned = df_uncleaned.drop_duplicates(subset=['Title'], keep='last')


In [9]:
df_ebay = df_uncleaned[["Title", "Price", "Reviews"]]


In [10]:
df = pd.concat([df_pr, df_yt, df_yt2, df_ebay])

In [11]:
# from nltk.corpus import stopwords
# en_stops = set(stopwords.words('english'))

# all_words = []

# for word in all_words: 
#     if word not in en_stops:
#         print(word)

In [12]:
# create graph dependency function with spacy
import spacy

nlp=spacy.load('en_core_web_sm')

def graph_dependency_spec_gen(texts_list):
    numerical_specifications = []
    qualitative_specifications = []

    for j, text in enumerate(texts_list): 
        doc = nlp(str(text))
        i = 0 
        while i < len(doc): 
            token = doc[i]
            spec = []
            if token.dep_ == "nummod" or token.dep_ == "amod": 
                flag = token.dep_ == "nummod"

                while not token.dep_ in ["dobj", "pobj", "appos"] : 
    #                 print(f"{token.text:{8}} {token.dep_+' =>':{10}}   {token.head.text:{9}}  {spacy.explain(token.dep_)} ")
                    if token.text not in en_stops: 
                        spec.append(token.text)
                    i += 1
                    if i >= len(doc): 
                        break
                    token = doc[i]
    #             print(f"{token.text:{8}} {token.dep_+' =>':{10}}   {token.head.text:{9}}  {spacy.explain(token.dep_)} ")
                spec.append(token.text)

                if flag: 
                    numerical_specifications.append([spec,str(text)])
                else: 
                    qualitative_specifications.append([spec,str(text)])

            else: 
                i+=1
    return numerical_specifications, qualitative_specifications
         

In [22]:
# df['Reviews'].explode().dropna().tolist()

['We recently had to replace our original heat pump hot water after 25 years. The Evoheat 270 is a well designed unit with great technology and a thoughtful accessible control panel. The installation went smoothly and the unit was up and running within minutes. Hot water within an hour. the inbuilt timer allows us to take best advantage of our solar power along with peak temperature control and vacation mode. Very happy so far after a week and a half- all running smoothly. Same Day Hot Water in Sydney were very helpful and informative about the product and installation options.',
 'Extremely happy with the service by provider on Good Friday.  Came to my property within 2 hours of calling them.  They installed an Evo270.  It has only been a few so far but it has proven to give excellent hot water and much better water flow to areas where I need it.',
 'Very happy with the information provided by Evoheat on there product. The plumber and the electrician that did the installation did a fa

In [23]:
df = df.explode('Reviews').dropna(subset=['Reviews'])

In [21]:
# create top words function

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.util import everygrams
from gensim.parsing.preprocessing import remove_stopwords , STOPWORDS
import itertools
from nltk.corpus import stopwords




# Create a Vectorizer Object
def topwords(doc): 
    en_stops = set(stopwords.words('english'))
    if type(doc[0]) == list: 
        doc = itertools.chain.from_iterable(doc)
    all_words = itertools.chain.from_iterable(map(lambda text: text.lower().split(), doc))
    all_words = filter(lambda word: word not in en_stops, all_words)

# vectorizer = CountVectorizer()

# vectorizer.fit(all_words) 
    counts = Counter(all_words)
    sorted_counts = sorted(counts, key=counts.get, reverse=True)
    return counts, sorted_counts
# Printing the identified Unique words along with their indices
# print("Vocabulary: ", sorted(vectorizer.vocabulary_, key=lambda x: vectorizer.vocabulary_.get(x)))
# vectorizer.vocabulary_['zloty']


In [26]:
en_stops = set(stopwords.words('english'))


In [ ]:
# sentimentanalyser class 
class SentimentAnalyser: 
    def __init__(self, dataframes, text_column='Reviews'):
        df = dataframes
        df = df.explode(text_column)
        df['Reviews'] = df['Reviews'].apply(lambda t: self.preprocess_text(str(t)))
        self.df = df
        self.labels = {
            'text_column': text_column, 
            'sentiment_analysis': "sentiment"
        }

    def preprocess_text(self, text):
        text = text.lower()                             # all lower case
        text = re.sub(r'\[.*?\]', ' ', text)            # remove text within [ ] (' ' instead of '')
        text = re.sub(r'\<.*?\>', ' ', text)            # remove text within < > (' ' instead of '')
        text = re.sub(r'http\S+', ' ', text)            # remove website ref http
        text = re.sub(r'www\S+', ' ', text)             # remove website ref www

        text = text.replace('€', 'euros')               # replace special character with words
        text = text.replace('£', 'gbp')                 # replace special character with words
        text = text.replace('$', 'dollar')              # replace special character with words
        text = text.replace('%', 'percent')             # replace special character with words
        text = text.replace('\n', ' ')                  # remove \n in text that has it

        text = text.replace('\'', '’')                  # standardise apostrophe
        text = text.replace('&#39;', '’')               # standardise apostrophe

        text = text.replace('’d', ' would')             # remove ’ (for would, should? could? had + PP?)
        text = text.replace('’s', ' is')                # remove ’ (for is, John's + N?)
        text = text.replace('’re', ' are')              # remove ’ (for are)
        text = text.replace('’ll', ' will')             # remove ’ (for will)
        text = text.replace('’ve', ' have')             # remove ’ (for have)
        text = text.replace('’m', ' am')                # remove ’ (for am)
        text = text.replace('can’t', 'can not')         # remove ’ (for can't)
        text = text.replace('won’t', 'will not')        # remove ’ (for won't)
        text = text.replace('n’t', ' not')              # remove ’ (for don't, doesn't)

        text = text.replace('’', ' ')                   # remove apostrophe (in general)
        text = text.replace('&quot;', ' ')              # remove quotation sign (in general)

        text = text.replace('cant', 'can not')          # typo 'can't' (note that cant is a proper word)
        text = text.replace('dont', 'do not')           # typo 'don't'

        text = re.sub(r'[^a-zA-Z0-9]', r' ', text)      # only alphanumeric left
        text = text.replace("   ", ' ')                 # remove triple empty space
        text = text.replace("  ", ' ')                  # remove double empty space

        tokens = TreebankWordTokenizer().tokenize(text)
        tokens = map(WordNetLemmatizer().lemmatize, tokens)
        preprocessed_text = " ".join(tokens)

        return preprocessed_text
        
    def Textblob(self): 
        df = self.df.copy()
        df[self.labels['sentiment_analysis']] = df[self.labels['text_column']].apply(lambda text: TextBlob(text).sentiment.polarity)
        return df 

    def BERTSentimentAnalysis(self): 
        df = self.df.copy() 
        classifier = pipeline("sentiment-analysis") 
        df[self.labels['sentiment_analysis']] = df[self.text_column].apply(lambda text: classifier(text)[0]['label'])
        return df 
        
    def ZeroShot(self, candidates=["sustainable", "unsustainable"], use_gpu=False): 
        if use_gpu: 
            device_idx = 0
        else: 
            device_idx = -1
        df = self.df_combined.copy() 
        # classifier = pipeline("zero-shot-classification", device=0)
        classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=device_idx)
        df[self.labels['zero_shot']] = df[self.text_column].apply(lambda text: classifier(text, candidate_labels=candidates)[0]['label']) 
        return df

    def combine_text(self,list_of_text):                     # define combine_text to take (list_of_text)
        combined_text = ' '.join(list_of_text)          # do this
        return combined_text  

    def get_topwords(self,data,dataframesgroupby_model=True): 

        extra_stopwords = ["wa", "doe", "ha", "video", "one", "subscribe", "channel", "watch", "watching", "thanks", "thank"] 
        all_stopwords = stopwords.words('english')
        all_stopwords.extend(extra_stopwords) 
        STOPWORDS.update(all_stopwords)

        '''create dtm'''
        cv = CountVectorizer(stop_words=all_stopwords)                       # initialise cv w/o stopwords
        data_cv = cv.fit_transform(data)                      # apply cv
        dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names(), index=pd.Series(self.df.iloc[:]))
        

        return dtm
    

In [51]:
results = [['btu', 'emission output', 'carbon dioxide equivalent', 'water conservation', 'gallon per minute', 'material reuse', 'source', 'wind', 'power', 'thermal mass', 'fuel', 'appliance', 'space'],
['timer', 'behavioral property', 'control', 'feedback', 'temperature', 'furnace', 'hot air', 'interior', 'home.'],
['solar powered', 'electric resistance', 'emission', 'heat pump', 'geothermal system', 'size', 'space', 'central location', 'evenly heat', 'wall', 'ceiling', 'drapery', 'window', 'door closed', 'space', 'climate', 'building material', 'factor', 'size', 'capacity', 'location'],
['solar powered', 'electric resistance', 'heat pumps', 'upfront cost', 'operating cost', 'maintenance cost']]

open_ai_keywords = []

for result in results: 
    open_ai_keywords += result


In [52]:
from transformers import pipeline 

sentiment_classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


2. Identify with keywords are mentioned the most

In [74]:
pos_counts = []
neg_counts = []

for search_term in open_ai_keywords: 
    pos_count = 0 
    neg_count = 0 
    for comment in df['Reviews'].to_list():     
        if search_term in comment: 
#             print(comment)
#             print("\n")
#             print(sentiment_classifier(comment))
            try:
                if sentiment_classifier(comment)[0]['label'] == 'POSITIVE': 
                    pos_count += 1 
                else: 
                    neg_count += 1
            except: 
                pass
    pos_counts.append(pos_count)
    neg_counts.append(neg_count)
    # print("search term: ", search_term)
    # print("positive: ", pos_count)
    # print("negative: ", neg_count)
    # print("\n")

search term:  btu
positive:  11
negative:  21


search term:  emission output
positive:  0
negative:  0


search term:  carbon dioxide equivalent
positive:  0
negative:  0


search term:  water conservation
positive:  0
negative:  0


search term:  gallon per minute
positive:  0
negative:  0


search term:  material reuse
positive:  0
negative:  0


search term:  source
positive:  82
negative:  120


search term:  wind
positive:  109
negative:  91


search term:  power
positive:  455
negative:  459


search term:  thermal mass
positive:  0
negative:  0


search term:  fuel
positive:  80
negative:  51


search term:  appliance
positive:  11
negative:  48


search term:  space
positive:  633
negative:  268


search term:  timer
positive:  161
negative:  98


search term:  behavioral property
positive:  0
negative:  0


search term:  control
positive:  402
negative:  182


search term:  feedback
positive:  63
negative:  12


search term:  temperature
positive:  564
negative:  333


search

In [75]:
sentiment_df = pd.DataFrame({
    "search term": open_ai_keywords, 
    "positive": pos_counts,
    "negative": neg_counts,
})

In [76]:
sentiment_df.drop_duplicates()

,search term,positive,negative
0,btu,11,21
1,emission output,0,0
2,carbon dioxide equivalent,0,0
3,water conservation,0,0
4,gallon per minute,0,0
5,material reuse,0,0
6,source,82,120
7,wind,109,91
8,power,455,459
9,thermal mass,0,0


In [77]:
# sort keywords mentioned by most positive 
# sort keywords mentioned by most negative
# sort keywards that has most negative over positive ratio

rank_positive =sentiment_df.sort_values(by='positive', ascending=False).drop_duplicates()
rank_negative =sentiment_df.sort_values(by='negative', ascending=False).drop_duplicates()
sentiment_df['neg pos ratio'] = sentiment_df['negative'] / sentiment_df['positive']
most_pain = sentiment_df.sort_values(by='neg pos ratio', ascending=False).drop_duplicates()



In [80]:
most_pain.drop_duplicates()

,search term,positive,negative,neg pos ratio
11,appliance,11,48,4.363636
35,door closed,2,6,3.000000
39,factor,12,36,3.000000
37,climate,4,12,3.000000
47,operating cost,1,2,2.000000
45,heat pumps,3,6,2.000000
0,btu,11,21,1.909091
6,source,82,120,1.463415
19,hot air,20,29,1.450000
25,heat pump,52,57,1.096154


In [81]:
# take the top 20 most mention keywords from most_pain

top_pain = most_pain[['search term','neg pos ratio']].head(20)
top_pain_words = top_pain['search term'].tolist()


In [82]:
def preprocess_text( text):
       text = text.lower()                             # all lower case
       text = re.sub(r'\[.*?\]', ' ', text)            # remove text within [ ] (' ' instead of '')
       text = re.sub(r'\<.*?\>', ' ', text)            # remove text within < > (' ' instead of '')
       text = re.sub(r'http\S+', ' ', text)            # remove website ref http
       text = re.sub(r'www\S+', ' ', text)             # remove website ref www

       text = text.replace('€', 'euros')               # replace special character with words
       text = text.replace('£', 'gbp')                 # replace special character with words
       text = text.replace('$', 'dollar')              # replace special character with words
       text = text.replace('%', 'percent')             # replace special character with words
       text = text.replace('\n', ' ')                  # remove \n in text that has it

       text = text.replace('\'', '’')                  # standardise apostrophe
       text = text.replace('&#39;', '’')               # standardise apostrophe

       text = text.replace('’d', ' would')             # remove ’ (for would, should? could? had + PP?)
       text = text.replace('’s', ' is')                # remove ’ (for is, John's + N?)
       text = text.replace('’re', ' are')              # remove ’ (for are)
       text = text.replace('’ll', ' will')             # remove ’ (for will)
       text = text.replace('’ve', ' have')             # remove ’ (for have)
       text = text.replace('’m', ' am')                # remove ’ (for am)
       text = text.replace('can’t', 'can not')         # remove ’ (for can't)
       text = text.replace('won’t', 'will not')        # remove ’ (for won't)
       text = text.replace('n’t', ' not')              # remove ’ (for don't, doesn't)

       text = text.replace('’', ' ')                   # remove apostrophe (in general)
       text = text.replace('&quot;', ' ')              # remove quotation sign (in general)

       text = text.replace('cant', 'can not')          # typo 'can't' (note that cant is a proper word)
       text = text.replace('dont', 'do not')           # typo 'don't'

       text = re.sub(r'[^a-zA-Z0-9]', r' ', text)      # only alphanumeric left
       text = text.replace("   ", ' ')                 # remove triple empty space
       text = text.replace("  ", ' ')                  # remove double empty space

       tokens = TreebankWordTokenizer().tokenize(text)
       tokens = list(map(WordNetLemmatizer().lemmatize, tokens))
       preprocessed_text = " ".join(tokens)

       return preprocessed_text

In [83]:
# extract all reviews from the dataset

all_reviews = df['Reviews'].tolist()


16351

In [84]:
# tokenize, leminizing (preprocessing) for top words
# do the same for the reviews
top_pain_key =[]
for word in top_pain_words: 
    top_pain_key.append(preprocess_text(word))

# top_pos_key =[]
# for word in top_pain_words: 
#     top_pos_key.append(preprocess_text(word))

all_reviews_clean =[]
for review in all_reviews:
    all_reviews_clean.append(preprocess_text(review))


16351

In [85]:
# for all the cleaned reviews, check if the top pain keywords are present, if it is, then store it for later
# same for positive keywords
pain_comments = {}


for all_review in all_reviews_clean: 
    for neg_key in top_pain_key: 
        if neg_key in all_review: 
            if neg_key not in pain_comments: 
                pain_comments[neg_key] = [all_review]
            else:
                if all_review not in pain_comments[neg_key]: 
                    pain_comments[neg_key].append(all_review)
                





In [87]:
# pain_comments

{'heat pump': ['we recently had to replace our original heat pump hot water after 25 year the evoheat 270 is a well designed unit with great technology and a thoughtful accessible control panel the installation went smoothly and the unit wa up and running within minute hot water within an hour the inbuilt timer allows u to take best advantage of our solar power along with peak temperature control and vacation mode very happy so far after a week and a half all running smoothly same day hot water in sydney were very helpful and informative about the product and installation option',
  'i replaced my instant gas hot water heater with an evo270 to take advantage of a solar system i recently put on i chose evo primarily due to the smaller footprint than some of the other heat pump unit a space wa tight a well a it good review the installation by john at oxy plumbing in canberra went really smoothly the unit it working well and seems very well built',
  'i paid for a heat pump hws got told a

3. Sentiment Analysis

In [89]:
# for each comment in the stored list, do sentiment analysis, and store the result (score and label together with the comment)
score ={}

for key in pain_comments:
    score[key] = []
    for review in pain_comments[key]:
        try:
            sc = sentiment_classifier(review)
            score[key].append([review,sc[0]['score'],sc[0]['label']])
        except: 
            pass




In [92]:
score['heat pump']

[['we recently had to replace our original heat pump hot water after 25 year the evoheat 270 is a well designed unit with great technology and a thoughtful accessible control panel the installation went smoothly and the unit wa up and running within minute hot water within an hour the inbuilt timer allows u to take best advantage of our solar power along with peak temperature control and vacation mode very happy so far after a week and a half all running smoothly same day hot water in sydney were very helpful and informative about the product and installation option',
  0.9992940425872803,
  'POSITIVE'],
 ['i replaced my instant gas hot water heater with an evo270 to take advantage of a solar system i recently put on i chose evo primarily due to the smaller footprint than some of the other heat pump unit a space wa tight a well a it good review the installation by john at oxy plumbing in canberra went really smoothly the unit it working well and seems very well built',
  0.999221920967

In [105]:
#split the comments into positive and negative dictionary
# the key of the dict is the keywords, and the value is the (comment, score and label(positive/negative))

score_pos = {}
score_neg = {}

for key in score.keys():
    score_pos[key] = []
    score_neg[key] = []
    for d in score[key]:
        if d[2] == 'POSITIVE':
            score_pos[key].append(d)
        else:
            score_neg[key].append(d)



In [109]:
#sort all the list in descending order from the dictionary
#highest score (negative/positive first)

from operator import itemgetter

for k,v in score_neg.items():
    score_neg[k] = sorted(v, key=itemgetter(1), reverse=True)
for k,v in score_pos.items():
    score_pos[k] = sorted(v, key=itemgetter(1), reverse=True)
# score_neg

{'heat pump': [['it keep breaking and is expensive to fix the heat pump section is broken and rheem sent a technician who could not fix it it wa running off the electric element and using three time the power but at least we had hot water my last technician did something to it and now it is completely dead i am still waiting for rheem to get it touch with a solution',
   0.9997352957725525,
   'NEGATIVE'],
  ['extremely over priced extremely noisy purchased the system in 2017 and now it dead i can not be clear enough on how noisy it is do not buy this product dollar3000 and only lasted 3 yr and it only come with a 12 month warranty their service fee and call out charge are ridiculous for the love of god do not spend money on a heat pump',
   0.9995947480201721,
   'NEGATIVE'],
  ['i paid for a heat pump hws got told an installer would call to organise installation after 6 week i called them back because i had heard nothing i got told that they could not install in south australia becau

In [144]:
# make another dict that only store comments ( and make sure not repeated)
top_neg_all ={}
for k,v in score_neg.items():
    top_neg_all[k] = []
    for i in score_neg[k][:100]:
        if i[0] not in top_neg_all[k]:
           top_neg_all[k].append(i[0])

top_pos_all ={}
for k,v in score_pos.items():
    top_pos_all[k] = []
    for i in score_pos[k][:100]:
        if i[0] not in top_pos_all[k]:
           top_pos_all[k].append(i[0])


# top_neg_all

{'heat pump': ['it keep breaking and is expensive to fix the heat pump section is broken and rheem sent a technician who could not fix it it wa running off the electric element and using three time the power but at least we had hot water my last technician did something to it and now it is completely dead i am still waiting for rheem to get it touch with a solution',
  'extremely over priced extremely noisy purchased the system in 2017 and now it dead i can not be clear enough on how noisy it is do not buy this product dollar3000 and only lasted 3 yr and it only come with a 12 month warranty their service fee and call out charge are ridiculous for the love of god do not spend money on a heat pump',
  'i paid for a heat pump hws got told an installer would call to organise installation after 6 week i called them back because i had heard nothing i got told that they could not install in south australia because they were not approved to give rebate i am waiting for a refund do not trust t

4.Graph dependency


In [145]:
# now we have all the top negative / positive comment in the order
# we run graph dependency on each comment 
# output is all the numerical and qualitative data extract from those top positive/negative comments

gp_neg_all ={}

for k,v in top_neg_all.items():
    gp_neg_all[k] = graph_dependency_spec_gen(top_neg_all[k])
   

In [171]:
to_pop = []
for k,v in gp_neg_all.items():
    if len(v) == 0:
        to_pop.append(k)

for i in to_pop:
    gp_neg_all.pop(i)
gp_neg_all.keys()



dict_keys(['heat pump', 'power', 'timer', 'temperature', 'wind', 'window', 'factor', 'operating cost', 'location', 'source', 'ceiling', 'door closed', 'wall', 'appliance', 'fuel', 'interior', 'climate', 'hot air', 'btu'])

In [ ]:
print(gp_neg_all['appliance'][0][4])
print( gp_neg_all['appliance'][0][5])
print(gp_neg_all['heat pump'][0][6])
print(gp_neg_all['heat pump'][0][1])
print(gp_neg_all['appliance'][0][18])
print(gp_neg_all['heat pump'][0][3])

5.OpenAI keywords extraction and summarization

In [218]:
# get qualitative respone from OpenAI
b = gp_neg_all['heat pump']
quan_b_com = [i[1] for i in b[0]]
print('all reviews that has quantitative data')
# print(quan_b_com)



all reviews that has quantitative data
['it keep breaking and is expensive to fix the heat pump section is broken and rheem sent a technician who could not fix it it wa running off the electric element and using three time the power but at least we had hot water my last technician did something to it and now it is completely dead i am still waiting for rheem to get it touch with a solution', 'extremely over priced extremely noisy purchased the system in 2017 and now it dead i can not be clear enough on how noisy it is do not buy this product dollar3000 and only lasted 3 yr and it only come with a 12 month warranty their service fee and call out charge are ridiculous for the love of god do not spend money on a heat pump', 'i paid for a heat pump hws got told an installer would call to organise installation after 6 week i called them back because i had heard nothing i got told that they could not install in south australia because they were not approved to give rebate i am waiting for a 

In [222]:
import os
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt= "Extract keywords from this text: "+ '.'.join(quan_b_com[:20]) +" ",
  temperature=1,
  max_tokens=100,
  top_p=1.0,
  frequency_penalty=1.0,
  presence_penalty=1.0
)
result = response.choices[0].text.replace('\n', ' ')
result

'  Broken heat pump, expensive to fix, noisy, short warranty'

In [223]:
# Summarry for designing for sustainable heater
import os
import openai
import time

openai.api_key = os.getenv('OPENAI_API_KEY')

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt=  'heater, ' + '.'.join(quan_b_com[1:20]) + "\n\nTl;dr:\n\n", # the first 20 sentences of the comments
  temperature=0.3,
  max_tokens=500,
  top_p=1,
  frequency_penalty=1,
  presence_penalty=1
)
e= response.choices[0]['text'].replace("\n", "")
e

"1. Don't buy Rheem heat pumps. They're overpriced and don't last very long.2. Beechwood is terrible at customer service and their products are also overpriced and don't last very long.3. Midea makes a good product but their customer service is terrible."